## Prepare Counter (Inner) Puzzle for Singleton

In [1]:
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)
import singleton_utils

from chia.types.blockchain_format.program import Program
from clvm_tools.binutils import disassemble

counter_puzzle = load_program("counter.clsp", ["../../../shared"])
terminate_singleton_puzzle = load_program("terminate-singleton.clsp", ["../../../shared"])

MOD = counter_puzzle
AMOUNT = 113 # odd amount
MAX_COUNT = 3
PUZZLE_HASH = 0x900ddeed
START_COUNT = 0
TERMINAL_PUZZLE = terminate_singleton_puzzle.curry(PUZZLE_HASH, AMOUNT)
puzzle = counter_puzzle.curry(
    MOD,
    MAX_COUNT,
    TERMINAL_PUZZLE,
    AMOUNT,
    START_COUNT
)
print_puzzle(puzzle, 60)

... 0x00900ddeed) (c (q . 113) 1))) (c (q . 113) (c (q) 1))))))


## Prepare Network & Standard TXN Coin

In [2]:
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')

standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

alice balance:		2000000000000
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3


## Prepare Launcher CoinSpend

In [3]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

launcher_coin = Coin(
    standard_txn_coin.name(), 
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    AMOUNT
)
launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    puzzle, # counter puzzle
)

launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        AMOUNT,
        [
            ("name", "a"), 
            ("start_count", START_COUNT)
        ]
    ]
)
launcher_announcement = launcher_solution.get_tree_hash()

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())


launcher id: bb94201a4ac3ca98c688e42f0b65d3dce0bbf648ce56e58280685a304f785a60
{
    "coin": {
        "amount": 113,
        "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
    "solution": "0xffa09fb24418ad304910f91fc91b5c80d429c14c8cf30f6570571189ef3a626d3bc6ff71ffffff846e616d6561ffff8b73746172745f636f756e74808080"
}


## Prepare Standard TXN CoinSpend & Signature

In [4]:
from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.program import Program
from chia.types.condition_opcodes import ConditionOpcode
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

standard_txn_coin_conditions = [
    # create launcher coin with the odd_amount (odd)
    Program.to(
        [
            ConditionOpcode.CREATE_COIN,
            singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH,
            AMOUNT,
        ]),
    # assert launcher coin announcement
    Program.to(
        [
            ConditionOpcode.ASSERT_COIN_ANNOUNCEMENT, 
            std_hash(launcher_id + launcher_announcement)
        ]),
]
if standard_txn_coin.amount > AMOUNT:
    standard_txn_coin_conditions = [
        *standard_txn_coin_conditions, 
        Program.to(
        [
            ConditionOpcode.CREATE_COIN,
            standard_txn_coin.puzzle_hash,
            standard_txn_coin.amount - AMOUNT,
        ])
    ]


delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(standard_txn_coin_conditions)
solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(standard_txn_coin_conditions)

standard_txn_coin_spend = CoinSpend(
    standard_txn_coin,
    standard_txn_puzzle,
    solution
)

standard_coin_message = (
    delegated_puzzle.get_tree_hash()
    + standard_txn_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)


# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

print_json(standard_txn_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff7180ffff3dffa05d11d325dc9afbe908d9089f41

## Create First Singleton Counter

In [5]:
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    [standard_txn_coin_spend, launcher_coin_spend],
    signature
)
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

additions:
{'amount': 113,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9'}
{'amount': 1749999999887,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3'}
{'amount': 113,
 'parent_coin_info': '0xbb94201a4ac3ca98c688e42f0b65d3dce0bbf648ce56e58280685a304f785a60',
 'puzzle_hash': '0x9fb24418ad304910f91fc91b5c80d429c14c8cf30f6570571189ef3a626d3bc6'}
removals:
{'amount': 1750000000000,
 'parent_coin_info': '0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001',
 'puzzle_hash': '0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3'}
{'amount': 113,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13

## Verify Singleton & Launcher Coin Records

In [6]:
launcher_cr = await singleton_utils.get_launcher_coin_record(
    get_block_records, get_additions_and_removals, get_puzzle_and_solution, 
    0, 10, ("name", "a"))
print(launcher_cr)

singleton_cr = await singleton_utils.get_last_singleton_coin_record(get_coin_records_by_parent_ids, launcher_id)
print(singleton_cr)

{'coin': {'amount': 113,
          'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
          'puzzle_hash': '0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9'},
 'coinbase': False,
 'confirmed_block_index': 2,
 'spent_block_index': 2,
 'timestamp': 1}
{'coin': {'amount': 113,
          'parent_coin_info': '0xbb94201a4ac3ca98c688e42f0b65d3dce0bbf648ce56e58280685a304f785a60',
          'puzzle_hash': '0x9fb24418ad304910f91fc91b5c80d429c14c8cf30f6570571189ef3a626d3bc6'},
 'coinbase': False,
 'confirmed_block_index': 2,
 'spent_block_index': 0,
 'timestamp': 1}


## Spend Singleton And Retrieve Counts

In [7]:
async def get_current_count(singleton_cr, name):
    singleton_parent_cr = await get_coin_record_by_name(singleton_cr.coin.parent_coin_info)
    count = None
    if singleton_parent_cr != None:
        if singleton_parent_cr.coin.puzzle_hash == singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH:
            launcher_cr = await singleton_utils.get_launcher_coin_record(
                                get_block_records, get_additions_and_removals, get_puzzle_and_solution, 
                                0, 10, ("name", name)
            )
            coin_spent = await get_puzzle_and_solution(launcher_cr.coin.name(), launcher_cr.spent_block_index)
            kv_list = coin_spent.solution.to_program().at("rrf").as_iter()
            for kv in kv_list:
                k = kv.first()
                if k == "start_count":
                    v = kv.rest().as_int()
                    count = v
        else:
            coin_spent = await get_puzzle_and_solution(singleton_parent_cr.coin.name(), singleton_parent_cr.spent_block_index)
            print_program(coin_spent.puzzle_reveal.to_program())
            
            print("singleton")
    return count

current_count = await get_current_count(singleton_cr, "a")
current_count

0

In [8]:
network.sim.pass_blocks(1)
coin_spend = launcher_coin_spend # eve spend
lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(coin_spend)

# prepare puzzle reveal
puzzle = counter_puzzle.curry(
    MOD,
    MAX_COUNT,
    TERMINAL_PUZZLE,
    AMOUNT,
    current_count
)
singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
            singleton_struct,
            puzzle,
    )

singleton_coin = await singleton_utils.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id)
print(singleton_coin)


lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(launcher_coin_spend)

inner_solution = Program.to([])

coin_spend = singleton_utils.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')
print(f'\nsingleton_puzzle:\n{singleton_puzzle.get_tree_hash()}')

print('\nsingleton_coin_spend:')
print_json(singleton_coin_spend.to_json_dict())

{'amount': 113,
 'parent_coin_info': '0xbb94201a4ac3ca98c688e42f0b65d3dce0bbf648ce56e58280685a304f785a60',
 'puzzle_hash': '0x9fb24418ad304910f91fc91b5c80d429c14c8cf30f6570571189ef3a626d3bc6'}
singleton_coin:
{'amount': 113,
 'parent_coin_info': '0xbb94201a4ac3ca98c688e42f0b65d3dce0bbf648ce56e58280685a304f785a60',
 'puzzle_hash': '0x9fb24418ad304910f91fc91b5c80d429c14c8cf30f6570571189ef3a626d3bc6'}

lineage_proof:
{'amount': 113,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
80

singleton_puzzle:
9fb24418ad304910f91fc91b5c80d429c14c8cf30f6570571189ef3a626d3bc6

singleton_coin_spend:


NameError: name 'singleton_coin_spend' is not defined

In [ ]:
from blspy import (G2Element)
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    [coin_spend],
    G2Element()
)
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

In [10]:
singleton_cr = await singleton_utils.get_last_singleton_coin_record(get_coin_records_by_parent_ids, launcher_id)
print(singleton_cr)
current_count = await get_current_count(singleton_cr, "a")
current_count

{'coin': {'amount': 113,
          'parent_coin_info': '0xc7585cba34ae1b0e20cbcaeb55b20a654d869858c35114ca317054d43c1b93f1',
          'puzzle_hash': '0xd6c086ce7cb127c2843d6ad38d185860123e0d531af4404cbffd54fa642c7a19'},
 'coinbase': False,
 'confirmed_block_index': 4,
 'spent_block_index': 0,
 'timestamp': 1}
(a (q 2 (q 2 (i (logand 47 52) (q 4 (c 32 (c 47 ())) (c (a 62 (c 2 (c 5 (c (a 42 (c 2 (c 39 (c (a (i 119 (q 2 54 (c 2 (c 9 (c 87 (c (a 46 (c 2 (c 5 ()))) ()))))) (q . 29)) 1) (c (a (i 119 (q . -73) (q . 87)) 1) ()))))) (c 119 ()))))) (a 58 (c 2 (c 5 (c (a 11 95) (q ()))))))) (q 8)) 1) (c (q (((73 . 71) 2 . 51) (c . 1) 1 . 2) ((not 2 (i 5 (q 2 50 (c 2 (c 13 (c (sha256 60 (sha256 52 36) (sha256 60 (sha256 60 (sha256 52 44) 9) (sha256 60 11 (sha256 52 ())))) ())))) (q . 11)) 1) (a (i (all (= (strlen 5) 34) (= (strlen 11) 34) (> 23 (q . -1))) (q 11 5 11 23) (q 8)) 1) 2 (i 11 (q 2 (i (a 38 (c 2 (c 19 ()))) (q 2 (i (not 23) (q 2 (i (= -77 (q . -113)) (q 2 58 (c 2 (c 5 (c 27 (c 52 ())))